In [1]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier,
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
import itertools
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier,

In [2]:
df=pd.read_excel("Stopwords_Removed_Ticket_TypeWithoutClient.xlsx")

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79053 entries, 0 to 79052
Data columns (total 10 columns):
Incident: Number      79053 non-null int64
Opened Date           79053 non-null datetime64[ns]
Queue                 79053 non-null object
Category              79053 non-null object
Client Location       69515 non-null object
Template              47271 non-null object
Request Definition    28023 non-null object
Short Description     79053 non-null object
Ticket_Type           79053 non-null object
Description1          78956 non-null object
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 6.0+ MB


In [49]:
df.dropna(subset=["Description1"],inplace=True)

In [50]:
X_train, X_test, y_train, y_test =  train_test_split(df['Description1'], df['Ticket_Type'], test_size=0.20, random_state=42)

In [51]:
my_vector = CountVectorizer(ngram_range=(1,2))
X_train_counts = my_vector.fit_transform(X_train)

In [52]:
tf_transformer = TfidfTransformer(use_idf=True,).fit(X_train_counts)
X_train= tf_transformer.transform(X_train_counts)

In [53]:
X_test_counts = my_vector.transform(X_test)

In [54]:
X_test = tf_transformer.transform(X_test_counts)

In [55]:
ad=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', max_depth=1),n_estimators=10,learning_rate=1)

In [56]:
ad.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1, n_estimators=10, random_state=None)

In [57]:
pred=ad.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ",accuracy_score(pred,y_test))

[[10048   222]
 [ 3523  1999]]
             precision    recall  f1-score   support

   Incident       0.74      0.98      0.84     10270
    Request       0.90      0.36      0.52      5522

avg / total       0.80      0.76      0.73     15792

Accuracy :  0.762854609929


In [13]:
X_train.shape

(63164, 241096)

In [15]:
X_test.shape

(15792, 241096)

In [ ]:
gb=GradientBoostingClassifier(loss='deviance',n_estimators=100,learning_rate=0.01,max_depth=1)

In [20]:
gb.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [25]:
pred=gb.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ",accuracy_score(pred,y_test))

[[10012   258]
 [ 2355  3167]]
             precision    recall  f1-score   support

   Incident       0.81      0.97      0.88     10270
    Request       0.92      0.57      0.71      5522

avg / total       0.85      0.83      0.82     15792

Accuracy :  0.834536474164


In [27]:
rand=RandomForestClassifier(n_estimators=20,criterion='entropy')

In [28]:
rand.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
pred=rand.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ",accuracy_score(pred,y_test))

[[9837  433]
 [1170 4352]]
             precision    recall  f1-score   support

   Incident       0.89      0.96      0.92     10270
    Request       0.91      0.79      0.84      5522

avg / total       0.90      0.90      0.90     15792

Accuracy :  0.898492907801


In [67]:
clf = XGBClassifier(max_depth=5,random_state=1)

In [68]:
clf.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=1,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [69]:
pred=clf.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ",accuracy_score(pred,y_test))

[[10030   240]
 [ 2167  3355]]
             precision    recall  f1-score   support

   Incident       0.82      0.98      0.89     10270
    Request       0.93      0.61      0.74      5522

avg / total       0.86      0.85      0.84     15792

Accuracy :  0.847581053698
